In [30]:
import pdb
import torch
from torch import nn
import cv2
import os
from PIL import Image
from skimage import io, transform
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from torchvision.datasets.folder import pil_loader
import random
import numpy as np
from torchsummary import summary
import pretrainedmodels
from sklearn.model_selection import KFold, StratifiedKFold


from albumentations import torch as AT
os.environ['CUDA_VISIBLE_DEVICES']=""
%matplotlib inline

In [2]:
!ls data

sample_submission.csv  test  train  train_labels.csv


In [3]:
train_df = pd.read_csv('data/train_labels.csv')
test_df = pd.read_csv('data/sample_submission.csv')

In [4]:
test_df.shape

(57458, 2)

In [5]:
train_df.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [6]:
train_df.at[0, 'label'] = 1

In [7]:
kfold = StratifiedKFold(20, shuffle=True, random_state=69)

In [8]:
kfold.split(train_df['id'], train_df['label'])

<generator object _BaseKFold.split at 0x7fe662ddc8b8>

In [9]:
train_df.iloc[test]

NameError: name 'test' is not defined

In [10]:
train_df.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,1
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [11]:
train_df.info(), test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220025 entries, 0 to 220024
Data columns (total 2 columns):
id       220025 non-null object
label    220025 non-null int64
dtypes: int64(1), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57458 entries, 0 to 57457
Data columns (total 2 columns):
id       57458 non-null object
label    57458 non-null int64
dtypes: int64(1), object(1)
memory usage: 897.9+ KB


(None, None)

In [12]:
%%time
img = mpimg.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

CPU times: user 5.96 ms, sys: 0 ns, total: 5.96 ms
Wall time: 17.8 ms


In [13]:
timeit mpimg.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

183 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
timeit pil_loader('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif')

In [ ]:
timeit cv2.imread('data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif', flags=cv2.IMREAD_COLOR)

In [ ]:
timeit io.imread("data/train/00001b2b5609af42ab0ab276dd4cd41c3e7745b5.tif")

In [15]:
img = torch.Tensor(img)

In [17]:
img.flip?

In [18]:
torch.flip?

##### use cv2.imread for reading images 

In [ ]:
img.shape


In [ ]:
plt.imshow(img)

In [ ]:
# plot some random training images with their ground truth labels
i = random.choice(range(train_df.shape[0]))
img = mpimg.imread('data/train/' + train_df.iloc[i]['id'] + '.tif')
plt.imshow(img)
plt.title(train_df.iloc[i]['label'])
plt.show()

In [ ]:
train_df['label'].value_counts()

In [ ]:
train_df['label'].hist

In [ ]:
img = mpimg.imread('data/train/dd6dfed324f9fcb6f93f46f32fc800f2ec196be2.tif')
plt.imshow(img)
plt.show();
aug_img = train_aug(image=img)
plt.imshow(aug_img['image'])
plt.show();

#### Dataloader analysis


In [ ]:
from dataloader import get_dataloader

In [ ]:
def unnormalize(img):
    mean = torch.as_tensor([0.485, 0.456, 0.406])
    std = torch.as_tensor([0.229, 0.224, 0.225])
    #return (img * std) + mean
    for t, m, s in zip(img, mean, std):    
        t.mul_(s).add_(m)
    return img

In [ ]:
phase = "train"
dataloader = get_dataloader(phase)

In [ ]:
batch = next(iter(dataloader))

In [ ]:
images, labels = batch
labels

In [ ]:
# plot some random training images with their ground truth labels
i = random.choice(range(len(images)))
img = images[i].clone()
img = unnormalize(img)
plt.imshow(img.permute(1, 2, 0))
plt.title(labels[i].item())
plt.show()

In [ ]:
mean = torch.as_tensor([0.485, 0.456, 0.406])
std = torch.as_tensor([0.229, 0.224, 0.225])
#return (img * std) + mean
#return img.mul_(std).add_(mean)
labels[i].item()

In [ ]:
mean, std, mean.shape, std.shape

In [9]:
import pretrainedmodels
pretrainedmodels.model_names

['fbresnet152',
 'bninception',
 'resnext101_32x4d',
 'resnext101_64x4d',
 'inceptionv4',
 'inceptionresnetv2',
 'alexnet',
 'densenet121',
 'densenet169',
 'densenet201',
 'densenet161',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnet101',
 'resnet152',
 'inceptionv3',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19_bn',
 'vgg19',
 'nasnetamobile',
 'nasnetalarge',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'dpn131',
 'dpn107',
 'xception',
 'senet154',
 'se_resnet50',
 'se_resnet101',
 'se_resnet152',
 'se_resnext50_32x4d',
 'se_resnext101_32x4d',
 'cafferesnet101',
 'pnasnet5large',
 'polynet']

In [20]:
# model_name='se_resnext50_32x4d'
model_name='nasnetamobile'

model = pretrainedmodels.__dict__[model_name](num_classes=1000)

In [16]:
model.features

TypeError: features() missing 1 required positional argument: 'input'

In [17]:
blocks = list(model.children())
# blocks[:-3]
model

NASNetAMobile(
  (conv0): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cell_stem_0): CellStem0(
    (conv_1x1): Sequential(
      (relu): ReLU()
      (conv): Conv2d(32, 11, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(11, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (comb_iter_0_left): BranchSeparables(
      (relu): ReLU()
      (separable_1): SeparableConv2d(
        (depthwise_conv2d): Conv2d(11, 11, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=11, bias=False)
        (pointwise_conv2d): Conv2d(11, 11, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (bn_sep_1): BatchNorm2d(11, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (separable_2): SeparableConv2d(
        (depthwise_conv2d): Conv2d(11, 11, kernel_size=(5, 5), stride=

In [ ]:
model.layer0.parameters()
model.layer1.parameters()
model.layer2.parameters()
model.layer3.parameters()
model.layer4.parameters()
#list(model.children())[0]#.parameterss()
model.avg_pool

In [ ]:
[
    {"params": model.layer0.paramters()},
    {"params": model.layer1.paramters()},
    {"params": model.layer2.paramters()},
    {"params": model.layer3.paramters()},
    {"params": model.layer4.paramters()},
    {"params": model.avg_pool.paramters()},
    {"params": model.last_linear.parameters(), "lr": 3e-5}
], lr=3e-5*0.001,

import torch

In [26]:
class classifier(torch.nn.Module):
    def __init__(self):
        self.gmp = torch.nn.functional.max_pool2d()
        self.aap = torch.nn.functional.adaptive_avg_pool2d()
        self.conc = torch.nn.conc()
        self.dp = torch.nn.Dropout(0.5)
        self.lin = torch.nn.Linear(1056, 1, bias=True) 
    def forward(self, x):
        pdb.set_trace()
        out1 = self.gmp(x)
        out2 = self.aap(x)
        flat = x.view(x.size[0], -1)
        out = self.conc(out1, out2, flat)
        out = self.dp(out)
        out = self.lin(out)
        return out

In [32]:
blocks = list(model.children())
head = torch.nn.Sequential(*list(model.children())[:-3])
# model._modules['layer0'] = head
#model._modules['avg_pool'] = torch.nn.AvgPool2d(kernel_size=5, stride=1, padding=0) 
#model._modules['avg_pool'] = torch.nn.Sequential(*[torch.nn.ConstantPad2d((0,1,0,1), 0), torch.nn.AvgPool2d(kernel_size=7, stride=1, padding=0)])
# model._modules['avg_pool'] = torch.nn.Sequential(torch.nn.AdaptiveAvgPool2d((1, 1)))
# model._modules['last_linear'] = torch.nn.Linear(in_features=2048, out_features=1, bias=True)
# model._modules['last_layer'] = classifier

In [12]:
torch.nn.AdaptiveAvgPool2d?

In [44]:
#head.parameters()
#for block in blocks:
base = list(blocks[0].children())[:-1] + blocks[1:-2]
torch.nn.Sequential(*base)
torch.nn.Dropout?
#    print(block)
#    print('*'*100)

In [46]:

torch.nn.Linear?

In [ ]:
#blocks[0] = head

In [ ]:
#blocks[-1] = torch.nn.Linear(in_features=2048, out_features=1, bias=True)

In [ ]:
#model = torch.nn.Sequential(*blocks)

In [ ]:
#model._modules

In [9]:
#img = torch.Tensor(1, 3, 224, 224)#.to('cuda')
img = torch.Tensor(1, 3, 112, 112)
img = torch.Tensor(1, 3, 96, 96)

In [15]:
out = model(img)

print(out)

tensor([[-0.2230]], grad_fn=<AddmmBackward>)


In [18]:
x = None
for param in model.children():
    
    print(param)
    if x is None:
        x = param(img)
    else:
        x = param(x)
    print(x.shape)

Sequential(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
)
torch.Size([1, 32, 47, 47])
CellStem0(
  (conv_1x1): Sequential(
    (relu): ReLU()
    (conv): Conv2d(32, 11, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(11, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (comb_iter_0_left): BranchSeparables(
    (relu): ReLU()
    (separable_1): SeparableConv2d(
      (depthwise_conv2d): Conv2d(11, 11, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=11, bias=False)
      (pointwise_conv2d): Conv2d(11, 11, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
    (bn_sep_1): BatchNorm2d(11, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (separable_2): SeparableConv2d(
      (depthwise_conv2d): Conv2d(11, 11, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=11, bias=False)
    

TypeError: forward() missing 1 required positional argument: 'x_stem_0'

In [26]:
# summary(model, input_size=(3, 96, 96))
# summary(model, input_size=(3, 112, 112))
summary(model.features, input_size=(3, 224, 224))

AttributeError: 'function' object has no attribute 'apply'

In [ ]:
torch.nn.functional.max_pool2d
torch.nn.functional.adaptive_avg_pool2d()

In [32]:
# torch.nn.Dropout?

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)




In [34]:
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained="imagenet")
backbone = model.features
classifier = nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)),
        Flatten(),
        nn.Dropout(0.5),
        nn.Linear(in_features=1056, out_features=1, bias=True)
        )



In [38]:
img = torch.Tensor(1, 3, 112, 112)
features = backbone(img)


RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 3

In [39]:
model.parameters()

<generator object Module.parameters at 0x7f961f946a20>

In [22]:
 
from albumentations import *

In [23]:
train_aug = Compose([
    Resize(224, 224),
    RandomRotate90(p=0.5),
    Transpose(p=0.5),
    Flip(p=0.5),
    OneOf([CLAHE(clip_limit=2), IAASharpen(), IAAEmboss(), RandomBrightnessContrast(), JpegCompression(), Blur(), GaussNoise() ], p=0.5), 
    HueSaturationValue(p=0.5), 
    ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=45, p=0.5),
    Normalize(p=1)
])
val_aug = Compose([
    Normalize(p=1)
])


In [24]:
ShiftScaleRotate?

In [25]:
Normalize?

In [ ]:
train_aug?

In [27]:
flip = Flip(p=1)

In [31]:
flip.apply?

##### AUC ROC

In [ ]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[1,1], random_state=1)
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = KNeighborsClassifier(n_neighbors=3)
model.fit(trainX, trainy)
# predict probabilities
probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
probs = probs[:, 1]


In [ ]:
testy.shape, probs.shape

In [ ]:
fpr, tpr, thresholds

In [ ]:
# calculate AUC
auc = roc_auc_score(testy, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(testy, probs)
# plot no skill
pyplot.plot([0, 1], [0, 1], linestyle='--')
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
# show the plot
pyplot.show()

In [ ]:
pyplot.savefig('lol.png')

In [ ]:
pyplot.close('all')

In [47]:
import apex

In [48]:
apex.amp.?

In [17]:
df1 = pd.read_csv('weights/ensemble/11Mar_nasnetamobile_fold0_ckpt17.csv')
df2 = pd.read_csv('weights/ensemble/9Mar_se_resnext50_32x4d_v3_fold0_1_2_allckpt17.csv')
df= df1.copy()
df['label'] = (df1['label'] + 3* df2['label'])/4

In [18]:
df.head()

,id,label
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0.031749
1,95596b92e5066c5c52466c90b69ff089b39f2737,0.046642
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0.001457
3,2c35657e312966e9294eac6841726ff3a748febf,0.005296
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0.030917


In [19]:
df.to_csv('weights/ensemble/resnextfold012ckpt17+nasnetmobileckpt17.csv', index=False)